In [1]:
import cv2
from ultralytics import YOLO
import numpy as np
import os
from collections import defaultdict
import torch
import torchvision
from collections import deque
import pandas as pd
os.environ['KMP_DUPLICATE_LIB_OK']='True'
#设置模型
model = YOLO("yolov8n.pt")

In [2]:
#读取视频
cap = cv2.VideoCapture("clip_1.mp4")
current_positions = {}
speeds = []
video_length_meters = 720  # 视频高度覆盖的真实长度，单位：米
video_resolution_height = 384 
conversion_factor = video_length_meters / video_resolution_height
# 视频高度的像素数
fps = 25 # 视频的fps
time_interval = 1 / fps #视频中一帧的现实时间差
frame_number = 0 #初始化帧数
prev_positions = {}  
centers = []
window_size = 25 
vehicle_window = deque(maxlen=window_size)
columns = ['Frame Number', 'Flow (vehicles/s)', 'Density (vehicles/m)']  # 初始列
df = pd.DataFrame(columns=columns)

In [3]:
def calculate_instantaneous_speed(current_position, previous_position, time_elapsed, center_y):
    if time_elapsed <= 0:  # 避免时间间隔为 0 或负数
        return 0
    # 计算两帧之间的像素距离
    pixel_distance = np.sqrt(
        (current_position[0] - previous_position[0]) ** 2 + (current_position[1] - previous_position[1]) ** 2)
    actual_distance = pixel_distance * conversion_factor
    max_y = video_resolution_height  
    # 当车辆在图像底部（靠近摄像头）时，进一步减小速度
    if center_y > max_y * 0.75:  # 车辆非常接近摄像头
        adjustment_factor = 70  # 增加缩放因子，减小速度
    elif center_y > max_y * 0.5:  # 车辆中等接近
        adjustment_factor = 30
    else:
        adjustment_factor = 5 + (max_y - center_y) / max_y * 15
    # 计算调整后的速度
    adjusted_speed = (actual_distance / time_elapsed) / adjustment_factor
    adjusted_speed = adjusted_speed / 6
    adjusted_speed = max(0, min(adjusted_speed, 36))
    return adjusted_speed

In [4]:
while True:
    success, frame= cap.read()
    if not success:
        break
    results = model.track(frame,persist=True)
    a = results[0].plot()
    track_ids = results[0].boxes.id.int().cpu().tolist()
    #####
    frame = int(frame_number)
    boxes = results[0].boxes.xywh.cpu() #获取车辆坐标
    current_frame_vehicle_count = len(track_ids)#当前帧的车辆数
    current_ids = set(track_ids)
    vehicle_window.append(current_ids)
    unique_id = set().union(*vehicle_window)#确保车辆id唯一
    flow_rate = len(unique_id) #车辆流量：单位时间内通过的车辆数
    vehicle_density = current_frame_vehicle_count #车辆密度
    print(f"Frame {frame_number}, Flow (last {window_size} frames): {flow_rate} vehicles, Density: {vehicle_density} vehicles")#输出当前帧数，车流量和车密度
    #######
    for box , track_id in zip(boxes,track_ids):
        x,y,w,h = box
        center_x, center_y = x,y #获取检测到的车辆中心点
        center = (float(center_x),float(center_y))
        centers.append(center)
        if track_id in prev_positions:
            prev_frame ,prev_center= prev_positions[track_id] #若检测到的车出现过，获取之前其出现的帧数和位置
            time_elapsed = (frame_number - prev_frame) * (1 / fps) #计算当前检测到的车辆的时间差
            try:
                speed = calculate_instantaneous_speed((float(center_x),float(center_y)),prev_center,time_elapsed,float(center_y)) #计算调整后的车速
            except ValueError:
                speed = 0
        else:
            speed = 0
        speeds.append(speed)
        prev_positions[track_id] = (frame_number,center)#更新车辆出现的帧数和位置

    #####
    row_data = [frame_number, flow_rate, vehicle_density] #保存数据
    for track_id in prev_positions.keys():
        if track_id in current_ids:
            # 如果车辆存在，记录当前速度
            idx = list(track_ids).index(track_id)
            speed1 = speeds[idx]
        else:
            # 如果车辆不在当前帧，将速度设置为0
            speed1 = None

        # 更新车辆位置并写入数据表
        vehicle_column = f'ID{track_id}'
        if vehicle_column not in df.columns:
            df[vehicle_column] = np.nan  # 初始化该列
        # 将速度信息加入数据
        row_data.append(speed1)
    # 确保数据表的列与row_data匹配
    while len(row_data) < len(df.columns):
        row_data.append(np.nan)  # 填充没有数据的列
    # 将数据写入当前帧的行
    df.loc[len(df)] = row_data
    frame_number +=1
    speeds = []
    cv2.imshow("123",a)
    if cv2.waitKey(1)& 0xFF ==ord('q'):
        break

cv2.destroyAllWindows()


0: 384x640 5 cars, 69.6ms
Speed: 7.5ms preprocess, 69.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 0, Flow (last 25 frames): 5 vehicles, Density: 5 vehicles

0: 384x640 5 cars, 40.2ms
Speed: 2.0ms preprocess, 40.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 1, Flow (last 25 frames): 5 vehicles, Density: 5 vehicles

0: 384x640 5 cars, 41.0ms
Speed: 1.0ms preprocess, 41.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 2, Flow (last 25 frames): 5 vehicles, Density: 5 vehicles

0: 384x640 6 cars, 38.6ms
Speed: 1.0ms preprocess, 38.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 3, Flow (last 25 frames): 6 vehicles, Density: 6 vehicles

0: 384x640 8 cars, 39.0ms
Speed: 1.0ms preprocess, 39.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 4, Flow (last 25 frames): 8 vehicles, Density: 8 vehicles

0: 384x640 8 cars, 29.0ms
Speed: 1.6ms preprocess, 29.0ms i

In [5]:
df.to_excel('a103.xlsx')#导出数据